In [1]:
import glob
import pandas as pd

raw_tsv_file = glob.glob('../raw_data/*.tsv')

In [73]:
df_arr = []
for fpath in raw_tsv_file:
    print(fpath)
    df_ = pd.read_csv(fpath, names=range(9))
    # df_.drop(columns=0, inplace=True)
    df_arr.append(df_)

../raw_data/Hau.tsv
../raw_data/Nguyen.tsv
../raw_data/outputs2.tsv
../raw_data/Khoi_4999.tsv
../raw_data/kiet-2023-03-22.tsv


In [90]:
df = pd.concat(df_arr)

In [91]:
df[3].replace({'NUMBER': 'PHONE', 
           'NUMBER_PREFIX': 'PHONE_PREFIX', 
           'AMOUT': 'AMOUNT',
           'REMAMONEY_PREFIX-': 'REMAMONEY_PREFIX',
           'FAX': 'OTHER',
           'FAX_PREFIX': 'OTHER'}, inplace=True)

In [92]:
df = df[~df[3].isin(['D', 'fa', 'a', 'ct'])]

In [93]:
df[~df[8].isna()]

,0,1,2,3,4,5,6,7,8
0,win_IMG_4492_00_jpg.rf.456d89e6d48331d0a49919e...,1454,4068,TDISCOUNT,1097,2687,1301,2758,"-43,200"
1,win_IMG_4492_00_jpg.rf.456d89e6d48331d0a49919e...,1454,4068,RECEMONEY_PREFIX,53,2778,554,2874,TIỀN KHÁCH TRẢ
2,win_IMG_4492_00_jpg.rf.456d89e6d48331d0a49919e...,1454,4068,RECEMONEY,1088,2776,1290,2856,"520,000"
3,win_IMG_4492_00_jpg.rf.456d89e6d48331d0a49919e...,1454,4068,RECEMONEY,1088,2880,1297,2952,"520,000"
4,win_IMG_4492_00_jpg.rf.456d89e6d48331d0a49919e...,1454,4068,REMAMONEY,1157,2994,1301,3065,"1,100"
...,...,...,...,...,...,...,...,...,...
4598,bhx_IMG_4372_00_jpg.rf.b7c5503b74d8e6593c8ab1b...,1732,4018,OTHER,163,3752,1511,3934,Quý khách vui lòng xem chi tiết Chính sách đổi...
4599,bhx_IMG_4372_00_jpg.rf.b7c5503b74d8e6593c8ab1b...,1732,4018,TITLE,404,612,1204,758,PHIẾU THANH TOÁN
4600,bhx_IMG_4372_00_jpg.rf.b7c5503b74d8e6593c8ab1b...,1732,4018,PRODUCT_NAME,83,2014,613,2096,SƯỜN NON HEO
4601,bhx_IMG_4372_00_jpg.rf.b7c5503b74d8e6593c8ab1b...,1732,4018,SHOP_NAME,470,107,1133,261,BÁCH HÓA XANH


In [94]:
threshold = 10
cnt = 0
removed_cols = []
for x, group in df.groupby(0):
    if len(group[group[8].isna()]) > len(group[8]) // 2:
        removed_cols.append(x)

In [95]:
df = df[~df[0].isin(removed_cols)]

In [97]:
df.to_csv('final.csv', index=False, header=['image','W','H','CLS','tlx', 'tly', 'brx', 'bry', 'text'])

In [98]:
df_train = df.sample(frac=0.8)

In [99]:
df_test = df.iloc[df.index.difference(df_train.index)]

In [101]:
df_train.to_csv('train.csv', index=False, header=['image','W','H','CLS','tlx', 'tly', 'brx', 'bry', 'text'])
df_test.to_csv('val.csv', index=False, header=['image','W','H','CLS','tlx', 'tly', 'brx', 'bry', 'text'])

In [25]:
df.to_csv('final.csv', index=False, header=['image','W','H','CLS','tlx', 'tly', 'brx', 'bry', 'text'])

ValueError: Writing 10 cols but got 9 aliases

In [103]:
import pandas as pd

df = pd.read_csv('final.csv')

In [104]:
df= df[~df.text.isna()]

In [105]:
list_file = list(df.groupby('image'))

In [106]:
import random
random.shuffle(list_file)
train=list_file[:int(len(list_file) * 0.8)]
test=list_file[int(len(list_file) * 0.8):]

In [110]:
new_df = {'size': [], 'ner_tags': [], 'tokens': [], 'bboxes': [], 'image': []}

for img_name, sample in test:
    new_df['size'].append(list(zip(sample['W'], sample['H'])))
    new_df['ner_tags'].append(list(sample['CLS']))
    new_df['bboxes'].append(list(zip(sample['tlx'],sample['tly'],sample['brx'],sample['bry'])))
    new_df['image'].append(img_name)
    new_df['tokens'].append(list(sample['text']))

In [111]:
new_df_ = pd.DataFrame(new_df)
new_df_.to_json('val.json', orient='records')

In [15]:
new_df = {'size': [], 'ner_tags': [], 'tokens': [], 'bboxes': [], 'image': []}

for img_name, sample in test:
    new_df['size'].append(list(zip(sample['W'], sample['H'])))
    new_df['ner_tags'].append(list(sample['CLS']))
    new_df['bboxes'].append(list(zip(sample['tlx'],sample['tly'],sample['brx'],sample['bry'])))
    new_df['image'].append(img_name)
    new_df['tokens'].append(list(sample['text']))
new_df_ = pd.DataFrame(new_df)
new_df_.to_json('val.json', orient='records')

In [112]:
df.CLS.unique()

array(['TDISCOUNT', 'RECEMONEY_PREFIX', 'RECEMONEY', 'REMAMONEY', 'OTHER',
       'PHONE_PREFIX', 'PHONE', 'SHOP_NAME', 'TAMOUNT_PREFIX',
       'SUB_TPRICE', 'DATETIME_PREFIX', 'UPRICE', 'UNIT',
       'PRODUCT_NAME_PREFIX', 'TPRICE_PREFIX', 'PRODUCT_NAME',
       'UNIT_PREFIX', 'ADDR', 'BILLID', 'SUB_TPRICE_PREFIX', 'AMOUNT',
       'TAMOUNT', 'CASHIER_PREFIX', 'BILLID_PREFIX', 'DATETIME', 'TITLE',
       'UPRICE_PREFIX', 'TPRICE', 'CASHIER', 'AMOUNT_PREFIX',
       'REMAMONEY_PREFIX', 'TDISCOUNT_PREFIX', 'FPRICE_PREFIX',
       'ADDR_PREFIX', 'FPRICE', 'UDISCOUNT_PREFIX', 'UDISCOUNT'],
      dtype=object)

In [121]:
train_1_path = '/Users/jaydentran1909/Pet Projects/bill-information-extraction/val.json'
train_2_path = '/Users/jaydentran1909/Pet Projects/bill-information-extraction/notebooks/val.json'

In [122]:
import json

In [123]:
with open(train_1_path, 'r') as f:
    train_1 = json.load(f)

In [124]:
with open(train_2_path, 'r') as f:
    train_2 = json.load(f)

In [125]:
final = train_1 + train_2

In [126]:
json_obj = json.dumps(final)

In [127]:
with open('final_val.json', 'w') as f:
    f.write(json_obj)

In [120]:
len(final)

758